In [1]:
import pickle
import spacy
import pandas as pd
import numpy as np

In [2]:
#!python -m spacy download en
nlp=spacy.load('en_core_web_sm')

# Getting the spacy doc phrases

In [4]:
dir='D:\escola\\tese\mycodes\gender_corpus\\raw_texts.pkl'
with open(dir, 'rb') as f:
    texts = pickle.load(f)

raw_texts are the texts collected from the wikipedia pages about the books -> list of strings where each string has several phrases that occur in a wikipedia paragraph. This list is formed by several (100) wikipedia pages. 

There are 

In [5]:
print('Example of one of the strings: \n ' , texts[1])
print()
print('There are {} strings (paragraphs) in raw_texts file'.format(len(texts)))


Example of one of the strings: 
  Frankenstein; or, The Modern Prometheus is an 1818 novel written by English author Mary Shelley. Frankenstein tells the story of Victor Frankenstein, a young scientist who creates a sapient creature in an unorthodox scientific experiment. Shelley started writing the story when she was 18, and the first edition was published anonymously in London on 1 January 1818, when she was 20. Her name first appeared in the second edition, which was published in Paris in 1821.

There are 5239 strings (paragraphs) in raw_texts file


Each of these strings/wikipedia paragraphs can be separated into phrases using spacy:

In [6]:
texts_span=[]
for paragraph in texts:
    phrase_gen=nlp(paragraph).sents #spacy generator
    for p in phrase_gen:
        texts_span.append(p)

KeyboardInterrupt: 

In [ ]:
texts_docs=[nlp(phrase.text) for phrase in texts_span]  #to make the phrases be a doc instead of a span (to save in pickle)

In [127]:
texts_docs[0:4]

[Frankenstein; or, The Modern Prometheus is an 1818 novel written by English author Mary Shelley.,
 Frankenstein tells the story of Victor Frankenstein, a young scientist who creates a sapient creature in an unorthodox scientific experiment.,
 Shelley started writing the story when she was 18, and the first edition was published anonymously in London on 1 January 1818, when she was 20.,
 Her name first appeared in the second edition, which was published in Paris in 1821.]

In [11]:
import pickle
with open('spacy_doc_from_raw_texts_wiki.pkl', 'wb') as f:
    pickle.dump(texts_docs,f)

# ========== The code can be run from here (the nlp () takes a bit to create the lists) =======

In [60]:
import pickle
import spacy
import pandas as pd
import numpy as np

In [61]:
#!python -m spacy download en
nlp=spacy.load('en_core_web_sm')

In [62]:

dir_='D:\escola\\tese\mycodes\gender_corpus\spacy_doc_from_raw_texts_wiki.pkl'
with open(dir_, 'rb') as f:
    texts_docs=pickle.load(f) 


In [63]:
print(texts_docs[1])
print(len(texts_docs))

Frankenstein tells the story of Victor Frankenstein, a young scientist who creates a sapient creature in an unorthodox scientific experiment.
19562


Using the spacy library, one can obtain several information about the words/tokens in a sentence:
+ morph - tells the person/number, tense of a verb (morphological characteristics of words)
    + tells the gender of a pronoun
    + etc.
+ dep_ - says what is the subj (nsubj or nsubjpass if tense=passive)
+ pos_ - part of speech is a simpler type of morph (just says PROPN, VERB etc)

In [64]:
print('token','sythatic function','part of speech','morph')
for i in texts_docs[1]:
    print(i,i.dep_,i.pos_,i.morph) 

print()

for ch in texts_docs[1][1].children: #i think that children gives the subject and object of a verb
    print(ch)


token sythatic function part of speech morph
Frankenstein nsubj PROPN Number=Sing
tells ROOT VERB Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
the det DET Definite=Def|PronType=Art
story dobj NOUN Number=Sing
of prep ADP 
Victor compound PROPN Number=Sing
Frankenstein pobj PROPN Number=Sing
, punct PUNCT PunctType=Comm
a det DET Definite=Ind|PronType=Art
young amod ADJ Degree=Pos
scientist appos NOUN Number=Sing
who nsubj PRON 
creates relcl VERB Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
a det DET Definite=Ind|PronType=Art
sapient compound ADJ Degree=Pos
creature dobj NOUN Number=Sing
in prep ADP 
an det DET Definite=Ind|PronType=Art
unorthodox amod ADJ Degree=Pos
scientific amod ADJ Degree=Pos
experiment pobj NOUN Number=Sing
. punct PUNCT PunctType=Peri

Frankenstein
story
.


Using the information above we:
+ find the root verbs using `token.dep_` and 
+ check if this verb is in the 3rd person singular using `token.morph.get('Person')==['3'] and token.morph.get('Number')==['Sing']`.
+ find the subject using the `token.children` of this root verb (it gives for example the subject and object of the verb) - generator which means that we need to create a loop if we want to get the words.

In [65]:
def get_roots_third_person(text):
    #text is a list of spacy docs that correspond to sentences

    third_person_phrases=[] #phrases in the 3rd person
    roots=[] # verbs that the model considers to be the verb of the independent sentence (root verb)
    root_childs=[] #subjects and objects of the root verb (?)

    for phrase in text:
        for token in phrase:
            if token.dep_=='ROOT':
                if token.morph.get('Person')==['3'] and token.morph.get('Number')==['Sing']:
                    third_person_phrases.append(phrase)
                    root_child=[word for word in token.children]
                    root_childs.append(root_child)
                    roots.append(token)
    return third_person_phrases,roots,root_childs
    

In [66]:
third_person_phrases,roots,root_childs=get_roots_third_person(texts_docs)
print(len(third_person_phrases))

7855


## Keeping only smaller sentences

Large sentences will be harder to learn for the computer (as they are harder to understand for humans). Some guidelines indicate that an average of 15-20 words are clearer phrases. 

https://techcomm.nz/Story?Action=View&Story_id=106#:~:text=A%20common%20plain%20English%20guideline,2009%3B%20Vincent%2C%202014) -> (Cutts, 2009; Plain English Campaign, 2015; Plain Language Association InterNational, 2015)(Cutts, 2009; Vincent, 2014).

Because of this, I excluded phrases that had more than 30 tokens (some tokens are punctuation so I left a bit of a buffer and this also allows for sentences that are a bit more complex) 

This step is a bit optional, but it removes almost 3000 sentences.

In [67]:
small_phrases=[]
for i in texts_docs:
    if len(i)<30:
        small_phrases.append(i)

len(small_phrases)

'''third_person_phrases_s=[]
for i in range(len(third_person_phrases:
    if len(i)<30:
        third_person_phrases_s.append(i)

len(third_person_phrases_s)

whould also work but we would but we lose the root verbs and childs of the function
'''

'third_person_phrases_s=[]\nfor i in range(len(third_person_phrases:\n    if len(i)<30:\n        third_person_phrases_s.append(i)\n\nlen(third_person_phrases_s)\n\nwhould also work but we would but we lose the root verbs and childs of the function\n'

In [68]:
third_person_phrases_small,roots_small,root_childs_small=get_roots_third_person(small_phrases)
print(len(third_person_phrases_small),'small 3rd person phrases')
print(len(list(set(third_person_phrases_small))),'small different 3rd person phrases')

print(roots_small[1],root_childs_small[1])
print(len(third_person_phrases_small[1]))




5040 small 3rd person phrases
5040 small different 3rd person phrases
is [Frankenstein, story, .]
10


In [82]:
range(len(third_person_phrases_small))

range(0, 4950)

I've also noticed that some phrases end with a ":" which means that they don't finish a thought.

In [97]:
l=third_person_phrases_small.copy()
for i in range(len(l)):
    if l[i][-1].text==':':
        third_person_phrases_small.remove(third_person_phrases_small[i])
print(len(third_person_phrases_small))

4941


Next, I tried to remove the phrases where the subject was "it".

In [98]:
n_neutral_small_phrases=[]
root_n_neutral=[]
root_childs_n_neutral=[]

for idx in range(len(third_person_phrases_small)):
    for child in root_childs_small[idx]:
        if 'nsubj' in child.dep_ and child.morph.get('Gender')!=['Neut']:
            n_neutral_small_phrases.append(third_person_phrases_small[idx])
            root_n_neutral.append(roots[idx])
            root_childs_n_neutral.append(root_childs_small[idx])
            
            
print(len(n_neutral_small_phrases)) #there are some where the subject is a name like "crew" which is still undefined
#there are also some (36) that are repeated but if i set the list the indexes change and i can't easity access the indexes in the rest of the lists.
#it is likely that in these cases spacy couldn't identify correctly the root verb and confused the subject
print(len(list(set(n_neutral_small_phrases))))

4543
4505


## Removing the sentences where "it" is the subject

I create a dataframe from the frases that dont have 'it' as the subject. This dataframe includes the phrase, root verb and the subject associated with it "root subject"

In [99]:
def highlight_word(phrase,colour_verb,colour_subj):
    # phrase is a doc string
    # words is a list of tokens (verb and subject)
    # colour is a string with rgb values (ex. 'rgb(155,217,230)')

    sent=[]
    root_verb=[root for root in phrase if root.dep_=='ROOT'][0]

    for token in phrase:
        if token.dep_=='ROOT':
            sent.append(" <span style='background: {}'>{}</span> ".format(colour_verb,token.text))
        elif 'nsubj' in token.dep_ and token in root_verb.children:
            sent.append(" <span style='background: {}'>{}</span> ".format(colour_subj,token.text))
        else:
            sent.append(token.text)

    
    return ' '.join(sent)

In [100]:
l=[]
for i in range(len(n_neutral_small_phrases)):
    phrase=n_neutral_small_phrases[i]
    root_verb=[root for root in phrase if root.dep_=='ROOT'][0]
    for token in phrase:
        if token.dep_=='ROOT' and token==root_verb:
            root=token
        elif 'nsubj' in token.dep_ and token in root_verb.children:
            subj=token
        
    l.append([phrase,root,subj])

non_neutral_df=pd.DataFrame(l)
non_neutral_df=non_neutral_df.rename(columns={0:"phrases",1:"root verb",2:'root subject'})
non_neutral_df

,phrases,root verb,root subject
0,"(Frankenstein, tells, the, story, of, Victor, ...",tells,Frankenstein
1,"(Frankenstein, is, a, frame, story, written, i...",is,Frankenstein
2,"(Robert, Walton, is, a, failed, writer, who, s...",is,Walton
3,"(During, the, voyage, ,, the, crew, spots, a, ...",spots,crew
4,"(A, few, hours, later, ,, the, crew, rescues, ...",rescues,crew
...,...,...,...
4538,"(The, overman, is, the, meaning, of, the, eart...",is,overman
4539,"(The, Superman, is, the, meaning, of, the, ear...",is,Superman
4540,"(Graham, Parkes, describes, his, own, 2005, tr...",describes,Parkes
4541,"(The, Overhuman, is, the, sense, of, the, eart...",is,Overhuman


There are also some (36) that are repeated but if i set the list the indexes change and i can't easity access the indexes in the rest of the lists. It is likely that in these cases spacy couldn't identify correctly the root verb and confused the subject

In [101]:
subject_list_n_neutral=[]
for i in root_childs_n_neutral:
    s=[]
    for w in i:
        if 'nsubj' in w.dep_:
            s.append(w)
    subject_list_n_neutral.append(s)

In [102]:
#indexes of the "strange" subjects
n=0
for idx in range(len(subject_list_n_neutral)):
    if len(subject_list_n_neutral[idx])>1:
        print(idx)
        n+=1
print(n)

448
449
534
535
549
550
569
570
723
724
754
755
824
825
935
936
1071
1072
1267
1268
1437
1438
1568
1569
1635
1636
1665
1666
1668
1669
1688
1689
1718
1719
1729
1730
2043
2044
2444
2445
2734
2964
2965
3077
3078
3080
3081
3088
3089
3215
3216
3343
3344
3350
3351
3650
3651
3896
3897
3911
3912
4007
4008
4064
4065
4081
4082
4131
4132
4151
4152
4279
4280
4363
4364
4421
4422
77


## Selecting the types of subjects "allowed"

One of the ways to reduce the list of sentences more is selecting which kinds of things can be a subject such as proper nouns (even though these may be for example names of countries or cities), pronouns such as she/he/her/etc., and some common nouns that are related to humans such as "the mother".
+ proper nouns: `token.pos_ == 'PROPN'`
+ she/he: `token.morph.get('PronType')==['Prs] and (token.morph.get('Gender')==['Masc'] or token.morph.get('Gender')==['Fem']`
+ there were only 2 phrases with her as the subject: "To be the stand-in motive for another's actions frightens her." and "Her "self-sacrifice is ultimately the greatest in the novel." so maybe this is not necessary (self-secrifice is the subj?)
+ common names: added to words to check (for example her father, etc - spacy identifies father as the subject in this case)

I also separated the phrases where the subject was a PROPN from the others because we may want to change these names from a predefined list.

In [103]:
print([txt for txt in non_neutral_df['root subject'][:100]])

[Frankenstein, Frankenstein, Walton, crew, crew, story, Victor, Victor, he, Victor, he, Victor, Victor, Victor, Creature, Creature, He, Creature, Creature, Victor, Creature, Clerval, Victor, He, He, Victor, Victor, Victor, Victor, Victor, father, Victor, ice, Walton, He, Victor, He, Victor, Walton, father, Mary, father, Ovid, This, Part, Blackwell, monster, monster, Reef, essay, Day, family, Prometheus, edition, Toro, He, Frankenstein, Frankenstein, Walton, crew, crew, story, Victor, Victor, he, Victor, he, Victor, Victor, Victor, Creature, Creature, He, Creature, Creature, Victor, Creature, Clerval, Victor, He, He, Victor, Victor, Victor, Victor, Victor, father, Victor, ice, Walton, He, Victor, He, Victor, Walton, father, Mary, father, Ovid, This]


In [104]:
idxs_PROPN=[i for i in range(len(non_neutral_df['root subject'])) if non_neutral_df['root subject'][i].pos_=='PROPN']
PROPN_df=non_neutral_df.loc[idxs_PROPN]

idxs_pron=[i for i in range(len(non_neutral_df['root subject'])) if (non_neutral_df['root subject'][i].morph.get('Gender')==['Masc'] or non_neutral_df['root subject'][i].morph.get('Gender')==['Fem'])]
pron_df=non_neutral_df.loc[idxs_pron]

In [105]:
PROPN_df

,phrases,root verb,root subject
0,"(Frankenstein, tells, the, story, of, Victor, ...",tells,Frankenstein
1,"(Frankenstein, is, a, frame, story, written, i...",is,Frankenstein
2,"(Robert, Walton, is, a, failed, writer, who, s...",is,Walton
7,"(From, a, young, age, ,, Victor, has, a, stron...",has,Victor
9,"(Victor, falls, ill, from, the, experience, an...",falls,Victor
...,...,...,...
4531,"(Nietzsche, pairs, this, word, with, its, oppo...",pairs,Nietzsche
4537,"(Martin, criticizes, Kaufmann, for, changing, ...",criticizes,Martin
4539,"(The, Superman, is, the, meaning, of, the, ear...",is,Superman
4540,"(Graham, Parkes, describes, his, own, 2005, tr...",describes,Parkes


In [106]:
pron_df

,phrases,root verb,root subject
8,"(At, the, university, ,, he, excels, at, chemi...",excels,he
10,"(After, a, four, -, month, recovery, ,, he, re...",receives,he
16,"(He, argues, that, as, a, living, being, ,, he...",argues,He
23,"(He, fears, that, the, female, will, hate, the...",fears,He
24,"(He, tears, apart, the, unfinished, female, cr...",tears,He
...,...,...,...
4493,"(With, the, help, of, a, fawn, who, has, also,...",makes,she
4494,"(Chapter, Four, –, Tweedledum, and, Tweedledee...",meets,She
4495,"(Finally, ,, the, brothers, begin, suiting, up...",predicts,She
4496,"(In, the, process, ,, he, introduces, Alice, t...",introduces,he


### Selecting the phrases with common names to see which are useful

In [107]:
idxs_nouns=[i for i in range(len(non_neutral_df['root subject'])) if non_neutral_df['root subject'][i].pos_=='NOUN']
nouns_df=non_neutral_df.loc[idxs_nouns]

In [108]:
nouns_df

,phrases,root verb,root subject
3,"(During, the, voyage, ,, the, crew, spots, a, ...",spots,crew
4,"(A, few, hours, later, ,, the, crew, rescues, ...",rescues,crew
5,"(The, recounted, story, serves, as, the, frame...",serves,story
6,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
14,"(Intelligent, and, articulate, ,, the, Creatur...",relates,Creature
...,...,...,...
4534,"(But, critical, opinion, varies, extremely, .)",varies,opinion
4535,"(The, book, is, "", a, masterpiece, of, literat...",is,book
4536,"(The, German, text, from, which, Hollingdale, ...",was,text
4538,"(The, overman, is, the, meaning, of, the, eart...",is,overman


In [109]:
PROPN_df['phrases'].index

Int64Index([   0,    1,    2,    7,    9,   11,   12,   13,   19,   21,
            ...
            4514, 4516, 4526, 4527, 4528, 4531, 4537, 4539, 4540, 4541],
           dtype='int64', length=1915)

I can't pickle the dataframes if there are a single token on the columns so I saved only the phrases and recreated the other two columns later

"_NotImplementedError: [E111] Pickling a token is not supported, because tokens are only views of the parent Doc and can't exist on their own. A pickled token would always have to include its Doc and Vocab, which has practically no advantage over pickling the parent Doc directly. So instead of pickling the token, pickle the Doc it belongs to._"

In [110]:
with open('PROPN_df_phrases.pkl', 'wb') as f:
    pickle.dump(PROPN_df['phrases'],f)

with open('pron_df_phrases.pkl', 'wb') as f:
    pickle.dump(pron_df['phrases'],f)

with open('nouns_df_phrases.pkl', 'wb') as f:
    pickle.dump(nouns_df['phrases'],f)

# ============================================================

In [3]:
import pickle
import pandas as pd

In [4]:

with open('PROPN_df_phrases.pkl', 'rb') as f:
    phrases_PROPN=pickle.load(f) 

with open('pron_df_phrases.pkl', 'rb') as f:
    phrases_pron=pickle.load(f) 

with open('nouns_df_phrases.pkl', 'rb') as f:
    phrases_nouns=pickle.load(f) 

In [5]:
def create_DF(phrase_series):
    l=[]
    for i in phrase_series.index:
        phrase=phrase_series[i]
        root_verb=[root for root in phrase if root.dep_=='ROOT'][0]
        for token in phrase:
            if token.dep_=='ROOT' and token==root_verb:
                root=token
            elif 'nsubj' in token.dep_ and token in root_verb.children:
                subj=token
        
        l.append([phrase,root,subj])
    l=pd.DataFrame(l)
    l=l.rename(columns={0:"phrases",1:"root verb",2:'root subject'})
    
    return l

In [6]:
PROPN_df=create_DF(phrases_PROPN)
pron_df=create_DF(phrases_pron)
nouns_df=create_DF(phrases_nouns)

In [7]:
print(len(PROPN_df),len(pron_df),len(nouns_df))

1915 768 1450


In [8]:
PROPN_df=PROPN_df.drop_duplicates()
pron_df=pron_df.drop_duplicates()
nouns_df=nouns_df.drop_duplicates()

print(len(PROPN_df),len(pron_df),len(nouns_df))

1899 762 1435


## Choosing the apparently useful subjects

With these datasets I chose which would be the nouns and proper nouns that could be useful for the dataset so that I could restrict a bit more the phrases to choose by hand. To do that I printed the list of subjects of the dataframes and chose by hand the ones that seem to be related to persons/characters.

In [9]:
useful_nouns_df_subject=['Victor','father','wife','girl','Fitzgerald','man','Dickens','Torvald','Tashtego','Starbuck','woman',
'mother','teacher','Farson', 'Browning','boy','sister','Utterson','Lanyon','boy','Jaggers','Estella','aunt','Jane','Rhys',
'Hester','Heart','charwoman','Marlow','Svidrigailov','Cassedy', 'Dostoevsky','Huck','Sanders','prince','Murry','Swinburne',
'Anatole','Deasy','Anatole','Chryses','Achilles','Diomedes','Hector','Thetis','Kleos','goddess','Agamemnon',
'Scrooge','Danglars','Douglass','king','Fantine','Thénardier','Marius','grandfather','Dostoevsky','Cervantes','Eurycleia',
'Penelope','Hobbes','Léonce','Reisz','Adèle','Chopin','Treatise','Locke','aunt','Vronsky','Levin','Eliza','Eva','husband',
'Crisóstomo','Elías','Fagin','Socrates','Cephalus','Pygmalion','Gyges','brother','Eliza','hero','heroine','Austen','Watson',
'woman','gentleman','Sissy','uncle','overman','Voltaire','Wolper']
#houskeeper? (it is neutral?) hunter leader carpenter narrator beggar bartender president precursor
#assistant character friend minister accountant manager clerk author 
#child
#Dracula is both the character and the book's name (see each phrase) - this may eventually also happen with Frankenstein
#maid is always feminin? priest always male?
#chairman/woman - maybe we can say "nouns that countain man/woman"  -> maybe this doesnt work because woman has the word man 
#Fate see if it is fate or a propnoun (appears with capslock - maybe im mistaking this with Faith); Hope
#soldier at the time this was a male almost for sure but now it isn't, keep it?

In [10]:
print(list(nouns_df['root subject']))

[crew, crew, story, Victor, Creature, Creature, Creature, Creature, Creature, Victor, father, ice, Victor, father, father, Part, monster, monster, essay, Day, family, edition, crew, crew, story, Victor, Creature, Creature, Creature, Creature, Creature, Victor, father, ice, Victor, father, father, Part, monster, monster, essay, Day, family, edition, Pride, wife, manner, housekeeper, novel, theme, Pride, Marriage, marriage, Inheritance, Pride, behaviour, sequel, girl, legacy, weather, entry, chapter, caterpillar, procession, biographer, critic, case, brand, Wonderland, scholar, binding, film, play, production, item, Fitzgerald, father, theme, neighbor, difference, difference, novel, work, contest, decision, repudiation, adaptation, episode, man, revelation, manuscript, jury, wife, edition, Resurrection, Resurrection, humour, opposite, Resurrection, sea, Darkness, shadow, blood, executioner, dictum, nanny, Torvald, Torvald, letter, Torvald, shift, story, sub, hunter, probity, protagonist,

In [11]:
nouns_df['root subject'].index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449],
           dtype='int64', length=1435)

In [12]:
idx_n=[]
for i in useful_nouns_df_subject:
    for ids in nouns_df['root subject'].index:
        if nouns_df['root subject'][ids].text==i:
            idx_n.append(ids)

noun_df_after=nouns_df.loc[idx_n]


In [13]:
noun_df_after

,phrases,root verb,root subject
3,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
9,"(Victor, suspects, that, the, Creature, is, fo...",suspects,Victor
12,"(Victor, dies, shortly, thereafter, ,, telling...",dies,Victor
25,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
31,"(Victor, suspects, that, the, Creature, is, fo...",suspects,Victor
...,...,...,...
1311,"(Her, uncle, ,, Mr, Peggotty, ,, finds, her, i...",finds,uncle
1448,"(The, overman, is, the, meaning, of, the, eart...",is,overman
1449,"(The, overman, is, the, meaning, of, the, eart...",is,overman
1324,"(Voltaire, ridicules, religion, ,, theologians...",ridicules,Voltaire


In [14]:
noun_df_after=noun_df_after.drop_duplicates()

In [15]:
noun_df_after

,phrases,root verb,root subject
3,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
9,"(Victor, suspects, that, the, Creature, is, fo...",suspects,Victor
12,"(Victor, dies, shortly, thereafter, ,, telling...",dies,Victor
25,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
31,"(Victor, suspects, that, the, Creature, is, fo...",suspects,Victor
...,...,...,...
1311,"(Her, uncle, ,, Mr, Peggotty, ,, finds, her, i...",finds,uncle
1448,"(The, overman, is, the, meaning, of, the, eart...",is,overman
1449,"(The, overman, is, the, meaning, of, the, eart...",is,overman
1324,"(Voltaire, ridicules, religion, ,, theologians...",ridicules,Voltaire


In [16]:
#Here we can see which are the phrases that contain the subject of our choice
name='Litz'
for i in range(len(list(PROPN_df['root subject']))):
    if list(PROPN_df['root subject'])[i].text==name:
        print(i)
        print(list(PROPN_df['phrases'])[i])


1851
Litz acknowledges the crudeness of these formulations and we recognize that he is attempting to discuss a quality of the novel which is hard to describe.


In [17]:
useful_nouns_df_subject=['Walton','Victor','Clerval','Mary','Blackwell','Toro','Bennet','Elizabeth','Collins','Darcy',
    'Charlotte','Jane','Catherine','Fitzwilliam','Wickham','Lydia','Bingley','Catherine','Austen','Joyce','Reynolds',
    'Alice','Tom','Jordan','Myrtle','Nick','Daisy','Fitzgerald','Gatsby','Wilde','George','Bechtel','Buchanan','Marx',
    'Levy','Dickens','Lorry','Evrémonde','Marquis','Gaspard','Carton','Darnay','Solomon','Defarge','Jerry','Manette',
    'Lucie','Defarge','Darnay','Carlyle','Simon','Törnqvist','Nora','Kristine','Krogstad','Torvald','Rank',
    'Nora','Kristine','Mencken','Stoddart','Dorial', 'Basil','Alan','James','Ishmael','Ahab','Queequeg','Stubb','Pip',
    'Dick','Bryant','Bezanson','Wright','Arvin','Matthiessen','Melville','Bryant','Milder','Gilman','Lanser','Treichler',
    'Horowitz','Edelstein','Harker','Lucy','Mina','Helsing','Showalter','Redmond','Enfield','Jekyll','Lanyon','Poole',
    'Utterson','Hyde','Wright','Havisham','Joe','Biddy','Herbert','Molly','Drummle','Jaggers','Magwitch','Jane','Reed',
    'Brocklehurst','Temple','Helen','Rochester','Rivers','John','Hester','Chillingworth','Pearl','Dimmesdale','Conrad',
    'Hochschild','Marlow','Nylander','Samsa','Sudau','Rubio','Drüke','Nabokov','Frank','Snitkina','Raskolnikov',
    'Marmeladov','Sonya','Razumikhin','Porfiry','Dunya','Mikolka','Luzhin','Lebezyatnikov','Svidrigailov','Dostoevsky',
    'Huck','Jim','Loftus','Polly','Twain','Hearn','Alberti','Finn','Eltis','Ellmann','Aynesworth', 'Wilde','Jack',
    'Bracknell','Gwendolen','Algernon','Gwendolen','Bracknell','Earnest','Foster','Edwards','Carby','Anderson',
    'Sanders','Machiavelli','Gilbert','Blake','Ulysses','Joyce','Stephen','Bloom','Mulligan','Gerty','Tolstoy','Pierre',
    'Boris','Rostov','Drubetskoy','Andrei','Denisov','Nikolai','Hélène','Pierre','Natasha','Rostov','Bolkonsky','Annenkov',
    'Strakhov','Dunnigan','Thoreau','Bagnall','Moore','Walden','Agamemnon','Chryses','Odysseus','Thetis','Aphrodite',
    'Athena','Diomedes','Zeus','Nestor','Poseidon','Polydamas','Hera','Achilles','Patroclus','Hephaestus','Hector','Priam',
    'Homer','Arnold','Pan','Barrie','Peter','Wendy','Robertson','Maimie','Darling','Lily','Bell','Hook','Smee','Faria',
    'Dantès','Mondego','Bertuccio','Carderousse','Andrea','Dorothy','Glinda','Henry','Oz','Beth','Jo','Meg','Laurence',
    'Brooke','Laurie', 'Meg', 'Beth','Amy','Lizzie','Saxton','Alcott','Carol','Scrooge','Dickens','Irving','Kelly','Jim',
    'Potter','Joe','Becky','Douglas','Hugo','Myriel','Valjean','Fantine','Javert','Thénardier','Marius','Cosette',
    'Gavroche','Enjolras','Perry','Karamazov','Pavlovich','Fyodorovich','Alyosha','Snegiryov','Smerdyakov','Grushenka',
    'Katerina','Zosima','Dmitri','Ilyusha','Alyosha','Snegiryov','Ivan','Kolya','Quixote','Fernando','Sancho','Anne',
    'Cervantes','Athena','Telemachus','Odysseus','Penelope','Hobbes','Doyle','Edna','Robert','Socrates','Mary','Locke',
    'Tully','Zuckert','Kenny','Emma','Knightley','Byrne','Henry','Karenin','Levin','Vronsky','Stiva','Bartlett',
    'Heathcliff','Dean','Earnshaw','Hindley','Catherine','Edgar','Cathy','Lockwood','Wiltshire','Scott','Alejandro',
    'Stewart','Alejandro','Eliza','Clare','Eva','Legree','Shelby','Dámaso','Crisóstomo','Guevarra','Salví','Tiago','Elías',
    'Guevarra','María','Twist','Oliver','Brownlow','Fagin','Nancy','Sikes','Bumble','Brownlow','Nancy','Rose','Bumble',
    'Bates','Buck','Mercedes','Thornton','Pizer','Gianquitto','Higgins','Eliza','Doolittle','Higgins','Socrates',
    'Polemarchus','Glaucon','Adeimantus','Plato','Poe','Gulliver','Mendez','Pedro','Jacobs','Brent','Martha','Jacobs',
    'Benjamin','William','Benny','Ellen','Flint','Bruce','John','Faustus','Lucifer','Marianne','Brandon','Steele',
    'Edward','Pollock','Favret','Holmes','Drebber','Crane','Rudkus','Jonas','Jurgis','Gradgrind','Bounderby','Stephen',
    'Sparsit','Tom','Blackpool','Louisa','Gradgrind','Harthouse','Cecilia','Bitzer','Rachael','Bazalgette','Mills','Pooh',
    'Hanyu','Charlie','Ashbee','Marcus','Mary','Weatherstaff','Fauntleroy','Gerzina','Masson','Burnett','Virgil','Dante',
    'Beatrice','David','Peggotty','Dora','Agnes','Hollington','Leavis','Needham','Bottiglia','Dexter',
    'Anne','Henrietta','Benwick','Clay','Russell','Croft','Isagani','Basilio','Simoun']

maybe_useful_nouns_df_subject=['Frankenstein','Mouse','Rabbit','Cat','Hatter','Queen','Turtle','Gardner','Duck',
    'Dormouse','Turtle','Pequod','Dracula','Witch','Lion','Ghost']


#some names are precedeed by miss & co - change this if the gender changes
#in Alice in wonderland's story there are several characters that are named after animals, maybe these will be useful
# because the phrases will work exchanging these names for human names - same for the wizard of oz

## Remove subjects that only appear once

I also concluded that if a name only appears one time probably is not related to the plot of the story but it is probably just a comment of someone that studied the text which is not what we want to a removed those names from the list.

In [18]:
def single_subj(list_of_subjects,original_dataframe):
    single=[]
    list_subj=list(original_dataframe['root subject'])
    for nn in list_of_subjects:
        n=0
        for i in range(len(list_subj)):
            if list_subj[i].text==nn:
                n+=1
        if n==1:
            single.append(nn)
    return single

Then I recovered the dataframe that had the subjects contained in the list of subjects.

In [19]:
idx_nouns=[]

for i in useful_nouns_df_subject:
    if i in single_subj(useful_nouns_df_subject,PROPN_df):
        useful_nouns_df_subject.remove(i)

for i in useful_nouns_df_subject:
    for idx in PROPN_df['root subject'].index:
        if PROPN_df['root subject'][idx].text==i:
            idx_nouns.append(idx)



propn_df_after=PROPN_df.loc[idx_nouns]


In [20]:
propn_df_after_keep=propn_df_after.copy() #this is in case something gets broken in propn_df_after we dont need to re-run the previous cell

## Remove duplicates

The following three dataframes are the ones that can be used to choose the phrases by hand:
+ noun_df_after - has 220 rows (some of which are prop nouns that were wrongly selected as common nouns by spacy)
+ propn_df_after - has 1144 rows (some don't correspond to the plot but to researchers commenting on the meaning of the plots)
+ pron_df - has 762 rows and are all the phrases that have "he/she" as the subject

In [21]:
noun_df_after=noun_df_after.drop_duplicates()
noun_df_after

,phrases,root verb,root subject
3,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
9,"(Victor, suspects, that, the, Creature, is, fo...",suspects,Victor
12,"(Victor, dies, shortly, thereafter, ,, telling...",dies,Victor
25,"(Victor, begins, by, telling, of, his, childho...",begins,Victor
31,"(Victor, suspects, that, the, Creature, is, fo...",suspects,Victor
...,...,...,...
1311,"(Her, uncle, ,, Mr, Peggotty, ,, finds, her, i...",finds,uncle
1448,"(The, overman, is, the, meaning, of, the, eart...",is,overman
1449,"(The, overman, is, the, meaning, of, the, eart...",is,overman
1324,"(Voltaire, ridicules, religion, ,, theologians...",ridicules,Voltaire


In [22]:
print(len(propn_df_after))
propn_df_after=propn_df_after.drop_duplicates()
propn_df_after

1459


,phrases,root verb,root subject
2,"(Robert, Walton, is, a, failed, writer, who, s...",is,Walton
16,"(At, the, end, of, Victor, 's, narrative, ,, C...",resumes,Walton
18,"(Walton, discovers, the, Creature, on, his, sh...",discovers,Walton
27,"(Robert, Walton, is, a, failed, writer, who, s...",is,Walton
41,"(At, the, end, of, Victor, 's, narrative, ,, C...",resumes,Walton
...,...,...,...
1883,"(Later, on, ,, Simoun, goes, to, Manila, and, ...",goes,Simoun
1886,"(Simoun, ,, driven, by, grief, ,, aborts, the,...",aborts,Simoun
1888,"(The, plan, thus, finalized, ,, Simoun, gives,...",gives,Simoun
1894,"(In, response, ,, Simoun, drinks, the, slow, -...",drinks,Simoun


For some reason some rows are duplicates but something in the doc/tokens is different and the .drop_duplicates() doesn't work:

In [23]:
print(pron_df['phrases'].loc[10])
print(pron_df['phrases'].loc[2])
print(type(pron_df['phrases'].loc[10]),type(pron_df['phrases'].loc[2]))
print(pron_df['phrases'].loc[10]==pron_df['phrases'].loc[2])
print(pron_df['phrases'].loc[10].text==pron_df['phrases'].loc[2].text)

He argues that as a living being, he has a right to happiness.
He argues that as a living being, he has a right to happiness.
<class 'spacy.tokens.doc.Doc'> <class 'spacy.tokens.doc.Doc'>
False
True


Because of this, I deleted them with the following function:

In [24]:
def equal_phrases(dataframe):

    same_index_2=[] #list of the duplicate phrase that appears in 2nd place

    for phrase1_idx in dataframe['phrases'].index:
        for phrase2_idx in dataframe['phrases'].index:
            if dataframe['phrases'].loc[phrase1_idx].text==dataframe['phrases'].loc[phrase2_idx].text and phrase2_idx!=phrase1_idx:
                same_index_2.append(max([phrase1_idx,phrase2_idx]))

    same_index_2=list(set(same_index_2))
    same_index_2.sort()
    
    return same_index_2


In [25]:
print('initial length: ', len(propn_df_after))
same_index_2=equal_phrases(propn_df_after)
propn_df_after.drop(index=same_index_2,axis=1,inplace=True)
print('after removing length: ', len(propn_df_after))

initial length:  1193
after removing length:  1173


In [26]:
print('initial length: ', len(noun_df_after))
same_index_2=equal_phrases(noun_df_after)
noun_df_after.drop(index=same_index_2,axis=1,inplace=True)
print('after removing length: ', len(noun_df_after))

initial length:  220
after removing length:  213


In [27]:
print('initial length: ', len(pron_df))
same_index_2=equal_phrases(pron_df)
pron_df.drop(index=same_index_2,axis=1,inplace=True)
print('after removing length: ', len(pron_df))

initial length:  762
after removing length:  754


## Colouring the subject and verb

To make the identification of the subjects easier the following function gives a version of a row with the subject and verb highlighted.

In [28]:
import re
def highlight_word(phrase,colour_verb,colour_subj):
    # phrase is a doc string
    # colour is a string with rgb values (ex. 'rgb(155,217,230)')

    sent=[]
    root_verb=[root for root in phrase if root.dep_=='ROOT'][0]

    for token in phrase:
        if token.dep_=='ROOT':
            sent.append(" <span style='background: {}'>{}</span> ".format(colour_verb,token.text))
        elif 'nsubj' in token.dep_ and token in root_verb.children:
            sent.append(" <span style='background: {}'>{}</span> ".format(colour_subj,token.text))
        else:
            sent.append(token.text)

    
    return ' '.join(sent)


def paint_words(dataframe_row):
    #receives a row of a dataframe (with ['phrases], ['root verb'] and ['root subject'] or just['phrases'] )
    txt=dataframe_row['phrases']

    colour_verb='rgb(25, 108, 56)'
    colour_subj='rgb(188, 108, 37)'

    sentence=highlight_word(txt,colour_verb,colour_subj)

    return sentence

Example of the result of the function

In [107]:
# new_prop_df 
# new_pronoun_df 
# new_noun_df 


from IPython.display import HTML
p=pron_df.loc[0]
display(HTML(paint_words(p)))


This function can be used to iterate over a dataframe and presents the highlighted phrase and a input where the user can say if the phrase is or not useful.

In [108]:
def select_useful(phrase):
    #receives a phrase (spacy doc) and returns if it is useful or not
    display(HTML(paint_words(phrase)))
    print('is it useful?[y/n \n /m - to see later \n /s - to stop]')
    ans=input()
    if ans.lower()!='y' and ans.lower()!='n' and ans.lower()!='m' and ans.lower()!='s':  
        print('Error: not allowed answer. Try again:')
        ans=input()

    return ans.lower()

In [161]:
list(propn_df_after.index).index(382)

263

In [172]:
list_indexes_ordered.index(1577)

996

In [173]:

v

dataF=propn_df_after
list_indexes_ordered=list(dataF.index)
list_indexes_ordered.sort()

for i in list_indexes_ordered[996:]:
    print(i)
    display(HTML(paint_words(dataF.loc[i])))
   

1577


1578


1580


1581


1582


...

1898


1899


1900


1901


1903


In [31]:
propn_df_after['phrases'][19]

Mary was just eighteen years old when she won the contest with her creation of Frankenstein.

In [29]:
indexes_propn_df_del=[19,21,24,79,80,81,88,93,110,111,112,114,115,116,117,118,119,144,145,149,170,171,173,174,175,177,
    207,217,250,251,252,253,254,255,256,258,261,262,263,264,267,268,270,273,275,343,349,361,384,387,400,401,404,406,407,
    410,411,459,478,480,500,514,515,522,525,531,532,534,537,539,563,566,568,571,572,574,575,576,577,578,579,
    580,581,582,583,585,586,589,590,591,592,593,595,597,601,604,605,606,609,614,615,616,621,622,629,630,631,643,651,653,
    654,655,656,695,700,702,703,704,711,713,718,732,738,745,747,754,755,759,778,779,780,781,782,783,786,791,799,800,804,
    811,895,900,906,909,1059,1073,1092,1093,1098,1100,1106,1111,1112,1113,1114,1117,1136,1154,1155,1156,1157,1161,1175,
    1195,1197,1198,1200,1211,1226,1230,1231,1272,1278,1297,1328,1347,1357,1373,1402,1405,1407,1425,1427,1428,1430,1466,
    1476,1480,1481,1483,1484,1485,1486,1488,1489,1492,1499,1503,1504,1520,1527,1529,1530,1532,1533,1535,1536,1537,1539,
    1542,1546,1547,1550,1560,1563,1564,1595,1666,1674,1676,1677,1678,1681,1683,1684,1686,1687,1691,1698,1699,1700,1701,
    1730,1734,1735,1749,1759,1762,1765,1768,1772,1776,1777,1783,1784,1786,1802,1866,1871,1898,1900]
#ver 475/6,

In [35]:
sentences_removed_propn=[]
sentences_removed_pron=[]
sentences_removed_noun=[]

for i in indexes_propn_df_del:
    sentences_removed_propn.append(propn_df_after['phrases'][i])

for i in indexes_pron_df_del:
    sentences_removed_pron.append(pron_df['phrases'][i])

for i in indexes_noun_df_del:
    sentences_removed_noun.append(noun_df_after['phrases'][i])




In [39]:
import pickle
with open('sentences_removed_before_pronoun.pkl', 'wb') as f:
    pickle.dump(sentences_removed_pron,f)

with open('sentences_removed_before_noun.pkl', 'wb') as f:
    pickle.dump(sentences_removed_noun,f)

with open('sentences_removed_before_propN.pkl', 'wb') as f:
    pickle.dump(sentences_removed_propn,f)

In [178]:
propn_df_after.drop(index=indexes_propn_df_del,axis=1,inplace=True)

In [179]:
propn_df_after

,phrases,root verb,root subject
2,"(Robert, Walton, is, a, failed, writer, who, s...",is,Walton
16,"(At, the, end, of, Victor, 's, narrative, ,, C...",resumes,Walton
18,"(Walton, discovers, the, Creature, on, his, sh...",discovers,Walton
3,"(From, a, young, age, ,, Victor, has, a, stron...",has,Victor
4,"(Victor, falls, ill, from, the, experience, an...",falls,Victor
...,...,...,...
1883,"(Later, on, ,, Simoun, goes, to, Manila, and, ...",goes,Simoun
1886,"(Simoun, ,, driven, by, grief, ,, aborts, the,...",aborts,Simoun
1888,"(The, plan, thus, finalized, ,, Simoun, gives,...",gives,Simoun
1894,"(In, response, ,, Simoun, drinks, the, slow, -...",drinks,Simoun


In [33]:
indexes_pron_df_del=[7, 60, 62,63,66,67,68,77,81,89,90,91,92,110,111,113,127,128,141,157,158,159,160,162,207,224,227,228,
229,230,232,233,235,236,242,246,252,262,269,275,281,292,293,294,299,304,305,306,307,315,316,319,321,334,358,361,368,374,
376,380,381,410,425,436,442,443,444,446,469,471,473,478,480,481,482,483,484,485,486,487,488,489,492,493,494,495,496,497,
510,517,518,519,520,533,541,543,568,584,485,591,593,595,497,598,599,600,601,602,603,606,607,609,608,610,611,613,624,625,
626,628,629,630,632,664,663,674,696,698,699,700,701,702,713,714,732,754,755,756,764,765,767]

In [140]:
pron_df.drop(index=indexes_pron_df_del,axis=1,inplace=True)

In [142]:
pron_df

,phrases,root verb,root subject
0,"(At, the, university, ,, he, excels, at, chemi...",excels,he
1,"(After, a, four, -, month, recovery, ,, he, re...",receives,he
2,"(He, argues, that, as, a, living, being, ,, he...",argues,He
3,"(He, fears, that, the, female, will, hate, the...",fears,He
4,"(He, tears, apart, the, unfinished, female, cr...",tears,He
...,...,...,...
760,"(He, then, spends, the, rest, of, his, days, h...",spends,He
761,"(She, also, observes, that, the, chess, pieces...",observes,She
762,"(There, she, forgets, all, nouns, ,, including...",forgets,she
763,"(With, the, help, of, a, fawn, who, has, also,...",makes,she


In [34]:
indexes_noun_df_del=[13,14,676,92,947,244,794,795,279,297,310,311,312,368,364,372,366,407,444,532,520,521,527,544,713,717,781,799,884,885,860,
1080,1082,1084,1085,1095,1081,1077,1078,277,278,1176,1185,1233,1448,1324,1339]



In [ ]:
noun_df_after.drop(index=indexes_noun_df_del,axis=1,inplace=True)

noun_df_after

After the deletion of the unuseful phrases

pron_df - 615 rows (he/she)

noun_df_after - 166 rows (some are PROPN other are father/sister/etc)

propn_df_after - 922 rows 

Dracula and Frankenstein (and Alice's cat, rabbit etc may also have useful phrases)

## Select useful for loop (painted)

Using the previous functions it is possible to select which are the phrases that are useful for the dataset by hand

In [56]:
#here the indexes will be saved to be used to select the rows from the dataframes using .loc[].
useful_phrases_idx=[]
not_useful_phrases_idx=[]
see_later_idx=[]

In [57]:

#change here the dataframe to use:
#  + pron_df
#  + propn_df_after
#  + noun_df_after

df=pron_df 

#here the indexes to use can be selected
phrases_from_df=df[:]


for i in list(phrases_from_df.index):
    ans=select_useful(phrases_from_df.loc[i])
    if ans=='y':
        useful_phrases_idx.append(i)
    if ans=='n':
        not_useful_phrases_idx.append(i)
    if ans=='m':
        see_later_idx.append(i)
    if ans=='s':
        break


print('last index:', i)
    #these indexes are used to recover the rows from the df with locs[useful_idx]

is it useful?[y/n 
 /m - to see later 
 /s - to stop]
last index: 0


In [251]:
print(useful_phrases_idx)
print()
print(not_useful_phrases_idx)
print()
print(see_later_idx)

[]

[]

[]
